In [29]:
import torch
from transformers import MobileBertTokenizer, MobileBertForMaskedLM, AutoTokenizer, AutoModelForMaskedLM
from rich import print

In [21]:
# Open the dataset file
import pandas as pd
df = pd.read_csv('../data/cleaned.csv')
df.head()

,ID,Content,Summary,Dataset,Summary_Tokens,Content_Tokens
0,f49ee725a0360aa6881ed1f7999cc531885dd06a,New York police are concerned drones could bec...,Police have investigated criminals who have ri...,CNN/Daily Mail,"['Police', 'have', 'investigated', 'criminal',...","['New', 'York', 'police', 'are', 'concerned', ..."
1,808fe317a53fbd3130c9b7563341a7eea6d15e94,By . Ryan Lipman . Perhaps Australian porn sta...,Porn star Angela White secretly filmed sex act...,CNN/Daily Mail,"['Porn', 'star', 'Angela', 'White', 'secretly'...","['By', '.', 'Ryan', 'Lipman', '.', 'Perhaps', ..."
2,98fd67bd343e58bc4e275bbb5a4ea454ec827c0d,"This was, Sergio Garcia conceded, much like be...",American draws inspiration from fellow country...,CNN/Daily Mail,"['American', 'draw', 'inspiration', 'from', 'f...","['This', 'was,', 'Sergio', 'Garcia', 'conceded..."
3,e12b5bd7056287049d9ec98e41dbb287bd19a981,An Ebola outbreak that began in Guinea four mo...,World Health Organisation: 635 infections and ...,CNN/Daily Mail,"['World', 'Health', 'Organisation:', '635', 'i...","['An', 'Ebola', 'outbreak', 'that', 'began', '..."
4,b83e8bcfcd51419849160e789b6658b21a9aedcd,By . Associated Press and Daily Mail Reporter ...,A sinkhole opened up at 5:15am this morning in...,CNN/Daily Mail,"['A', 'sinkhole', 'opened', 'up', 'at', '5:15a...","['By', '.', 'Associated', 'Press', 'and', 'Dai..."


In [13]:
# Load the pre-trained MobileBERT model and tokenizer
model = MobileBertForMaskedLM.from_pretrained('google/mobilebert-uncased')
tokenizer = MobileBertTokenizer.from_pretrained('google/mobilebert-uncased')

Some weights of the model checkpoint at google/mobilebert-uncased were not used when initializing MobileBertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing MobileBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MobileBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [26]:
def summarize_text(input_file):
    # Load the input text
    with open(input_file, 'r') as f:
        text = f.read()

    # Split the input text into chunks of 512 tokens
    chunk_size = 512
    chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

    # Initialize the MobileBERT model and tokenizer
    model_name = 'google/mobilebert-uncased'
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForMaskedLM.from_pretrained(model_name)

    # Generate the summary for each chunk
    summaries = []
    for chunk in chunks:
        # Tokenize the input text
        inputs = tokenizer(chunk, padding='max_length', truncation=True, max_length=512, return_tensors='pt')

        # Generate the summary using the MobileBERT model
        with torch.no_grad():
            output = model(input_ids=inputs.input_ids, attention_mask=inputs.attention_mask)
            mask_output = output[0][:, :len(chunk.split()), :]
            predicted_tokens = torch.argmax(mask_output, dim=-1)

        # Convert the predicted tokens back to text
        summary = tokenizer.decode(predicted_tokens[0], skip_special_tokens=True)
        summaries.append(summary)

    # Concatenate the summaries and return the final summary
    final_summary = ' '.join(summaries)
    return final_summary


In [30]:
# Example usage
print( summarize_text('input.txt') )

Some weights of the model checkpoint at google/mobilebert-uncased were not used when initializing MobileBertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing MobileBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MobileBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


. new york police are concerned drones could become tools for terrorists, and are investigating ways to stop 
potential attacks. until now police haven't acknowledged drones as a potential weapon, but the nypd has now said 
the technology has advanced enough that someone could use them to carry out an air assault using chemical weapons 
and firearms. police want to develop technology which will allow them to take control of drones as well as scan the
skies for them . aerial explosives are the number one threat as they investigate ways to stop attacks. deputy chief
salvatore dipace, however, was concerned about an incident last year where a drone was landed in front of german 
chancellor angela merkel and'could have taken the chancellor and her people out'a drone which was flown over a 
packed football stadium in manchester, england, just over a week ago, resulting in the suspected pilot being 
arrested. they . the bomb squad, emergency services and aviation units are working on a plan to counter weaponized 
drones. the nypd hasn't received any intelligence indicating there is an imminent threat, but has become 
increasingly concerned over the last year. deputy chief salvatore dipace told cbs news :'we've looked at some 
people that have jury - rigged these drones to carry guns, to carry different types of explosives if . we had also 
seen video showing how accurate an attack from a drone could be :'we've seen some video where the drone was flying 
at different targets along the route and very accurately hitting the targets with the paintball. the nypd now sees 
a drone carrying explosives as the number one threat. mr dipace's concerns follow an incident in germany last year 
where a drone was able to land just in front of german chancellor angela merkel as . as she delivered a speech, 
germany sparking fears that the device could easily be used to commit a terrorist act. he said :'if you really 
think about what could have happened there, the drone hit its target right on the mark and could have taken the 
president and her people out.'there has been a dramatic increase of incidents involving drones in new york city in 
the last year, with 40 reported. in some cases unmanned aircraft systems or drones had . one incident this summer, 
a drone which was almost 800 feet off the ground, nearly collided with a police helicopter. nypd aviation unit 
member, sergeant antonio hernandez said :'we're flying in the dark ; we have night - vision goggles on, we're 
trying to get a job

In [24]:
# # Save content column from row 1 to a text file
# text = df.iloc[0]['Content']
# with open('input.txt', 'w', encoding='utf-8') as f:
#     f.write(text)